# LSTM을 이용한 네이버 영화 리뷰 감정 분석

Data Download : https://github.com/e9t/nsmc/

- ratings_train.txt
- ratinns_test.txt

### 1. txt Dataset을 csv로 변환

In [1]:
import pandas as pd

columns = ['id', 'text', 'label']
train_data = pd.read_csv('../txt_datasets/ratings_train.txt', sep='\t', names=columns, skiprows=1).dropna() # null data 삭제
test_data = pd.read_csv('../txt_datasets/ratings_test.txt', sep='\t', names=columns, skiprows=1).dropna()

# csv 파일로 변환
train_data.to_csv('../csv_datasets/ratings_train.csv', index=False)
test_data.to_csv('../csv_datasets/ratings_test.csv', index=False)


print(train_data.head())

         id                                               text  label
0   9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1   3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2  10265843                                  너무재밓었다그래서보는것을추천한다      0
3   9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4   6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1


### 2. 전처리

In [2]:
import torch
from torchtext import data

SEED = 1234
torch.manual_seed(SEED)

- Field를 지정한다. 한글 데이터를 다루므로 토크나이저 또한 별도로 지정해야 한다.

In [3]:
from konlpy.tag import Okt  # 형태소 분석기중 하나인 Okt를 불러온다
okt = Okt()  # Okt 클래스의 인스턴스 생성. 이 인스턴스를 사용하여 텍스트를 형태소로 분리할 수 있다.

# torchtext에 내장된 데이터셋을 이용하는 게 아니므로, 각 컬럼별로 해당하는 Field를 지정해줘야 한다.
TEXT = data.Field(tokenize=okt.morphs, include_lengths=True) # data.Field는 텍스트 데이터를 어떻게 처리할지 정의한다. torkenize 인자로 Okt.morphs를 사용하고 있으므로, 각 문장을 형태소로 분리한다.
# include_lengths=True : 각 텍스트 데이터의 토큰화된 길이도 함께 반환하게 한다. 이 길이 정보를 패딩 작업에서 사용한다.
LABEL = data.LabelField(dtype = torch.float) # data.LabelField는 label을 어떻게 처리할지 정의한다.

# 위에서 정의한 TEXT와 LABEL은 이후 과정에서 데이터를 전처리하는데 사용된다.

- 데이터를 불러오고 검증 데이터를 추가한다.

In [4]:
# CSV 파일의 각 컬럼이 어떻게 처리될지를 정의하는 딕셔너리를 생성한다.
# 'text' 컬럼은 TEXT Field를 사용하여 처리되고, 'label' 컬럼은 LABEL 필드를 사용하여 처리된다.
fields = {'text' : ('text', TEXT), 'label' : ('label', LABEL)}

# dictionary 형식은 {csv 컬럼명 : (데이터 컬럼명, Field 이름)}

In [5]:
# TabularDataset 은 데이터를 불러오면서 필드에서 정의했던 토큰화 방법으로 토큰화를 수행한다.
train_data, test_data = data.TabularDataset.splits(
    path = '../csv_datasets',
    train = 'ratings_train.csv',
    test = 'ratings_test.csv',
    format = 'csv',
    fields = fields, 
)

In [6]:
# 불러온 데이터의 형태 확인
vars(train_data[0]), vars(train_data[1]), vars(train_data[2])

({'text': ['아', '더빙', '..', '진짜', '짜증나네요', '목소리'], 'label': '0'},
 {'text': ['흠',
   '...',
   '포스터',
   '보고',
   '초딩',
   '영화',
   '줄',
   '....',
   '오버',
   '연기',
   '조차',
   '가볍지',
   '않구나'],
  'label': '1'},
 {'text': ['너', '무재', '밓었', '다그', '래서', '보는것을', '추천', '한', '다'], 'label': '0'})

In [7]:
# 훈련 데이터의 갯수 확인
print("훈련 데이터 수 : {}".format(len(train_data)))
print("테스트 데이터 수 : {}".format(len(test_data)))

훈련 데이터 수 : 149995
테스트 데이터 수 : 49997


In [8]:
# 데이터에 검증 데이터가 따로 주어져 있지 않으므로 생성해준다.
import random
random.seed(SEED)
train_data, valid_data = train_data.split(random_state=random.seed(SEED))
# torchtext의 split 메서드는 기본적으로 데이터를 7:3 비율로 나눈다.

- 다음으로, 단어 벡터는 전처리된 단어 벡터를 받도록 한다. 
- 한글을 지원하는 fasttext.simple.300d 를 사용한다.
- 사전훈련된 단어집에 없는 단어는 0으로 처리하는 것을 방지하기 위해 unk_init = torch.Tensor.normal_ 옵션을 준다

In [9]:
MAX_VOCAB_SIZE = 35000  # 단어장의 크기를 35000개로 제한한다. 만약 단어장에 더 많은 단어가 있으면, 빈도 수가 낮은 단어부터 제외된다.

TEXT.build_vocab(  # 단어 집합 생성. 입력된 텍스트 데이터의 모든 고유 단어를 인덱스화하고 관련 정보 저장
    train_data,  # 어휘를 구축할 훈련 데이터
    max_size = MAX_VOCAB_SIZE,  # 빈도 수가 높은 단어 우선으로 포함시킨다
    vectors = 'fasttext.simple.300d',  # 사용할 단어 벡터를 지정하는 것. fasttext.simple.300d는 FastText에서 제공하는 300차원의 사전 훈련된 임베딩이다.
    unk_init = torch.Tensor.normal_ # 사전 훈련된 임베딩에 없는 단어를 초기화하는 방법을 지정
)

LABEL.build_vocab(train_data)  # label에 대한 어휘 사전 구축. 여기서는 label이 어떤 종류인지 정의하는데 사용된다(이진 분류, 다중 분류 등)

In [10]:
# TEXT, LABEL 단어장의 갯수 확인

print("TEXT 단어장의 갯수 : {}".format(len(TEXT.vocab)))
print("LABEL 단어장의 갯수 : {}".format(len(LABEL.vocab)))

# <unk>와 <pad> 토큰이 추가되어 있으므로 단어의 갯수가 35,002개이다. 
# <unk> : Unknown을 나타내며, 단어장에 없는 단어를 대체한다. train data에 등장하지 않은 단어가 test data에서 나타나면 해당 단어는 <unk>로 처리된다
# <pad> : Padding을 나타내며, 배치 처리를 위해 모든 문장을 동일한 길이로 만들어야 할 때, 짧은 문장에 패딩을 추가하기 위해 사용된다. 
# 패딩은 실제 의미를 가지지 않는 토큰으로 문장의 길이를 맞춘다. 

TEXT 단어장의 갯수 : 35002
LABEL 단어장의 갯수 : 2


In [11]:
# 단어와 인덱스 사이 매핑 확인

# itos(integer to string) : 정수 인덱스를 단어로 매핑하는 리스트
# 여기서 처음 10개의 요소를 출력하면, 단어장의 상위 10개 단어를 볼 수 있다.
print(TEXT.vocab.itos[:10]) 

# 단어를 정수로 매핑
# stoi(string to integer) : 단어를 정수 인덱스로 매핑하는 딕셔너리
# LABEL의 경우 레이블(클래스)에 대한 매핑을 나타낸다.
print(LABEL.vocab.stoi)

['<unk>', '<pad>', '.', '이', '영화', '의', '..', '가', '에', '을']
defaultdict(None, {'0': 0, '1': 1})


In [12]:
# 첫 번째 train 데이터 예시 출력
vars(train_data.examples[0])

{'text': ['야한', '장면', '기다리는것도', '곤욕', '이군'], 'label': '0'}

In [13]:
# 훈련 데이터에서 길이가 0인 텍스트 찾기
# 모든 훈련 데이터들 돌면서, 텍스트 길이가 0인 example의 인덱스를 출력한다

for i in range(len(train_data)):
    if len(train_data.examples[i].text)==0:print(i)

In [14]:
# BucketIterator를 이용하여 데이터 생성자를 만든다
# 데이터를 배치로 나누고 반복할 수 있는 iterator를 생성하는 과정
BATCH_SIZE = 64

device = 'cuda' if torch.cuda.is_available() else 'cpu'

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits( 
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch = True,
    device = device,
)

In [15]:
# train data의 첫 번째 배치에서 텍스트 부분을 가져온다
print(next(iter(train_iterator)).text)


# 패딩 제외 길이로 정렬된 (문장, 길이) 순의 데이터로 이루어져 있다.
# 첫 번째 텐서는 배치의 텍스트 부분이다. 각 숫자는 특정 단어를 나타내며, 단어장 내에서의 인덱스이다.
# 두 번째 텐서는 각 문장의 실제 길이를 나타낸다. 모든 문장의 실제 길이가 16으로 동일한 모습이다.

text_tensor, text_lengths = next(iter(train_iterator)).text
print(text_tensor.shape)  # 문장의 형태를 나타낸다. (이 배치에서 가장 긴 문장 길이, 배치 크기) 
# 실제 길이가 16인 문장들은 같은 배치에 모였고, 다른 배치들도 이와 유사한 방식으로 구성된다)
print(text_lengths.shape) # 문장 길이의 형태를 나타낸다. (배치 크기)

# BucketIterator는 배치 내의 문장을 가능한 같은 길이로 만들기 위해 패딩을 사용한다.
# 따라서 배치 내의 모든 문장은 같은 길이인 57로 만들어진다.
# 만약 실제 문장의 길이가 57보다 작다면, 패딩 토큰이 추가되어 문장의 길이가 57가 된다. 


(tensor([[   60,   266,     0,  ...,    60,   480,   205],
        [   25,   188,   553,  ...,   494, 22321,     4],
        [  532,    67,  2539,  ...,   146,  5656,     5],
        ...,
        [    0,     7,  1479,  ...,   866,  1107,   286],
        [ 5769,     0,  7947,  ...,    46,    25,    44],
        [   62,     2,  1414,  ...,  5570,  1149,   199]], device='cuda:0'), tensor([16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16], device='cuda:0'))
torch.Size([57, 64])
torch.Size([64])


### 3.  모델 생성

- Multi-layer bi-directional LSTM을 써서 모델을 생성한다. 
- drop out 적용
- nn.utils.rnn.packed_padded_sequence 써서 패킹/ 언패킹 처리를 할 것이다   

- 패킹 : 다른 길이를 가진 여러 시퀀스를 하나의 패딩된 텐서로 묶는 과정. 시퀀스의 길이를 맞추기 위해 패딩 토큰을 추가할 수 있다

- ex) [1,2,3] [4,5] ----> 패딩 후 : [1,2,3] [4,5,0]

- 최종적인 hidden의 size는 [num layers * num directions, batch size, hidden dim] 이다.
- 구체적으로 [forward_layer_0, backward_layer_0, forward_layer_1, bachward_layer_1, ..., forward_layer_n, backward_layer_n]의 형태로 출력된다.
- 꼭대기층의 hidden만 필요로 하므로, hidden[-2,:,:]과 hidden[-1,:,:]만 뽑아서 concatenate할 것이다.

In [16]:
import torch.nn as nn

- nn.Embedding의 padding_idx 옵션 : 특정 인덱스에 해당하는 임베딩 벡터가 항상 0 벡터가 되도록 설정한다.
- 패딩을 나타내는 토큰에 사용되며, 패딩이 모델에 정보를 추가하지 않도록 한다

In [17]:
emb = nn.Embedding(3,5,padding_idx=1)  # 3: 임베딩 레이어에 입력될 가능한 토큰의 갯수, 5: 각 토큰을 5차원의 벡터로 변환
test = torch.tensor([0,1,2])

In [18]:
print(emb(test)) # padding_idx에 해당하는 벡터는 0으로 나온다.(인덱스 1에 해당하는 패딩 벡터 )

tensor([[ 0.4467,  0.6607,  0.0908,  0.4816,  1.4613],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.4984,  0.1401, -0.1989,  0.1570, -0.4236]],
       grad_fn=<EmbeddingBackward>)


모델 구현

In [19]:
# def print_shape(name, data):
#     print(f'{name} has shape {data.shape}')

In [20]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout, pad_idx):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)  # TEXT를 밀집 벡터로 변환한다.  padding_idx=pad_idx : 패딩 토큰에 해당하는 벡터가 0이 되도록 지정
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout) # embedding_dim 차원의 입력을 받아 hidden_dim 차원의 hidden state를 출력한다.
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # 최종 출력을 생성하는 FC Layer. 양 방향 LSTM이므로 hidden_dim * 2를 입력 차원으로 사용한다
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        # text = [sent_len, batch_size]
        # print_shape('text',text)
        
        embedded = self.dropout(self.embedding(text))  # 텍스트가 임베딩 레이어를 통과하여 밀집 벡터로 변환되고,  dropout이 적용된다
        # embedded = [sent_len, batch_size, emb_dim]
        # print_shape('embedded', embedded)
        
        # pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)  # 패딩된 시퀀스를 패킹한다. 이는 계산 효율성을 높이기 위해 사용되며, 길이가 다른 여러 시퀀스를 하나의 배치로 효과적으로 처리한다.
        packed_output, (hidden, cell) = self.rnn(packed_embedded)  # 패킹된 임베딩이 LSTM 레이어를 통과하며, 이 과정에서 입력 시퀀스의 순서 정보가 학습된다.output, hidden state, cell state를 얻으며, hidden state와 cell state는 각각 시퀀스 내의 중간 정보를 저장한다. 
        # print_shape('packed_output', packed_output)
        # print_shape('hidden', hidden)
        # print_shape('cell', cell)
        
        # unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)  # 언패킹 과정에서 패킹된 출력은 다시 원래 형태로 변환된다.
        # print_shape('output', output)
        # print_shape('output_lengths', output_lengths)
        
        
        # output = [sent_len. batch_size, hidden_dim * num_directions]
        # output over padding tokens are zero tensors
        # hidden = [num_layers * num_directions, batch_size, hidden_dim]
        # cell = [num_layers * num_directions, batch_size, hidden_dim]
        
        # concat the final forward and backward hidden layers
        # and apply drop-out
        
        # print_shape('hidden[-2, :, :]', hidden[-2, :, :])
        # print_shape('hidden[-1, :, :]', hidden[-1, :, :])
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        # 마지막 hidden state는 정방향 및 역방향 LSTM의 출력을 연결하여 생성된다. 연결된 hidden state에 drop-out을 적용한다
        # print_shape('hidden', hidden)
        # hidden = [batch_size, hidden_dim * num_directions]
        
        res = self.fc(hidden)  # 최종 hidden state가 FC layer를 통과하여 최종 출력을 생성한다.
        # print_shape('res',res)
        
        return res 
        
        

In [21]:
# hyper parameter 설정

INPUT_DIM = len(TEXT.vocab)  # 단어장에 있는 고유한 단어의 수
EMBEDDING_DIM = 300   # fasttxt dim과 동일하게 설정
HIDDEN_DIM = 256  # hidden state의 크기
OUTPUT_DIM = 1  # 모델의 최종 출력 크기
N_LAYERS = 2  # 층의 수
BIDIRECTIONAL = True  # 양방향 여부
DROPOUT = 0.5  # Drop 확률
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]  # 패딩 인덱스로, 패딩 토큰에 해당하는 인덱스이다. 

In [22]:
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

In [23]:
# 모델의 파라미터 갯수 세보기
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("이 모델은 {:,} 개의 파라미터를 가지고 있다".format(count_parameters(model)))

이 모델은 13,220,857 개의 파라미터를 가지고 있다


사전 학습된 fasttext모델의 단어 벡터를 embedding 레이어에 복사하여 담는다

In [24]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([35002, 300])


In [25]:
model.embedding.weight.data.shape

torch.Size([35002, 300])

In [26]:
model.embedding.weight.data.copy_(pretrained_embeddings)  # copy_ 메서드는 인수를 현재 모델의 웨이트에 복사한다

tensor([[-0.1117, -0.4966,  0.1631,  ..., -1.4447,  0.8402, -0.8668],
        [ 0.1032, -1.6268,  0.5729,  ...,  0.3180, -0.1626, -0.0417],
        [ 0.0569, -0.0520,  0.2733,  ..., -0.0695, -0.1606, -0.0989],
        ...,
        [-0.2952,  0.8295, -0.9815,  ..., -0.2008, -0.9005, -0.7168],
        [ 1.0138, -0.5817, -0.3477,  ..., -0.4448,  0.4386, -1.0933],
        [ 0.5348, -0.9496, -1.9084,  ...,  0.4537,  1.2611,  2.0677]])

In [27]:
# 여기서 <unk>와 <pad>는 수동으로 0벡터로 만든다

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
UNK_IDX, PAD_IDX

(0, 1)

In [28]:
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0569, -0.0520,  0.2733,  ..., -0.0695, -0.1606, -0.0989],
        ...,
        [-0.2952,  0.8295, -0.9815,  ..., -0.2008, -0.9005, -0.7168],
        [ 1.0138, -0.5817, -0.3477,  ..., -0.4448,  0.4386, -1.0933],
        [ 0.5348, -0.9496, -1.9084,  ...,  0.4537,  1.2611,  2.0677]])


- pad는 pad_idx 옵션 때문에 훈련 내내 0으로 남아있겠지만, unk는 학습될 것이다.

### 4. 모델 훈련

In [29]:
import torch.optim as optim

# optimizer
optimizer = optim.Adam(model.parameters())

# loss function : binary cross entropy with logits : 임의의 실수를 입력으로 받아서 sigmoid 함수를 취해 0과 1 사이의 값으로 변환한 뒤, label과의 binary cross entropy를 계산한다
criterion = nn.BCEWithLogitsLoss()

In [30]:
# 모델과 손실함수를 GPU에 올린다

model = model.to(device)
criterion = criterion.to(device)

In [31]:
# 평가를 위해 임의의 실수를 0과 1 두 정수 중 하나로 변환하는 함수를 만든다

def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

Train function 만든다. 현재 batch.text는 (토큰들, 문장 길이)로 구성되어 있으니 분리한다

In [32]:
def train(model, iterator, optimizer, critertion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        text, text_lengths = batch.text
        text_lengths = text_lengths.cpu()
        predictions = model(text, text_lengths).squeeze(1)
        # print_shape('predictions', predictions)
        
        loss = critertion(predictions, batch.label)
        # print_shape('loss', loss)
        
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

- 평가를 위한 함수는 그래디언트 업데이트를 하지 않아야 하므로 with torch.no_grad(): 구문으로 감싼다.
- dropout을 평가때는 적용하지 않아야 하므로 model.eval()을 넣어준다

In [36]:
def evaluate(model, iterator, critertion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator :
            text, text_lengths = batch.text
            text_lengths = text_lengths.cpu()
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


에폭마다 걸린 훈련시간을 측정하는 함수를 만든다

In [37]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Train

In [40]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'checkpoint/2.2_LSTM_Sentiment_Analysis.pt')
        
    print("Epoch : {}/{} | Epoch time : {}m {}s".format(epoch+1, N_EPOCHS, epoch_mins, epoch_secs))
    print("train Loss : {} | Train Acc : {} %".format(train_loss, train_acc * 100))
    print(" Val  Loss : {} |  Val  Acc : {} %".format(valid_loss, valid_acc * 100))

Epoch : 1/10 | Epoch time : 0m 36s
train Loss : 0.21319543356206416 | Train Acc : 91.18286190710005 %
 Val  Loss : 0.36098812095058913 |  Val  Acc : 86.40739143910733 %
Epoch : 2/10 | Epoch time : 0m 35s
train Loss : 0.20477817703712684 | Train Acc : 91.59186895049105 %
 Val  Loss : 0.3757393475283276 |  Val  Acc : 86.53421774506569 %
Epoch : 3/10 | Epoch time : 0m 35s
train Loss : 0.1958646133614705 | Train Acc : 91.96458798875756 %
 Val  Loss : 0.38614889237479394 |  Val  Acc : 86.47017045454545 %
Epoch : 4/10 | Epoch time : 0m 35s
train Loss : 0.18982169231187637 | Train Acc : 92.31741739656051 %
 Val  Loss : 0.40746685677335004 |  Val  Acc : 86.41658635302024 %
Epoch : 5/10 | Epoch time : 0m 35s
train Loss : 0.18129242627355363 | Train Acc : 92.66453382084096 %
 Val  Loss : 0.4211816519223662 |  Val  Acc : 86.45875609733842 %
Epoch : 6/10 | Epoch time : 0m 35s
train Loss : 0.17565286343736775 | Train Acc : 92.88850379091875 %
 Val  Loss : 0.41308987893121824 |  Val  Acc : 86.518681

Test 데이터 Loss, Accuracy 출력

In [42]:
# save한 모델 불러오기
model.load_state_dict(torch.load('checkpoint/2.2_LSTM_Sentiment_Analysis.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(" Test Loss : {} |  Test Acc : {} %".format(test_loss, test_acc * 100))

 Test Loss : 0.366023244417232 |  Test Acc : 85.96562685259163 %


### 5. 사용자 데이터 사용

predict_semtiment 함수를 만든다.

이 함수의 기능 :
- sets the model to evaluation mode
- tokenizes the sentence, i.e. splits it from a raw string into a list of tokens
- indexes the tokens by converting them into their integer representation from our vocabulary
- gets the length of our sequence
- converts the indexes, which are a Python list into a PyTorch tensor
- add a batch dimension by unsqueezeing
- converts the length into a tensor
- squashes the output prediction from a real number between 0 and 1 with the sigmoid function
- converts the tensor holding a single value into an integer with the item() method

In [45]:
from konlpy.tag import Okt
okt = Okt()

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok for tok in okt.morphs(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)  # 배치
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    
    return prediction.item()

In [46]:
predict_sentiment(model, "영화 매우 재밌네")

0.9852145314216614

In [47]:
predict_sentiment(model, "개노잼 ㅋㅋ")

0.0010542043019086123

In [74]:
predict_sentiment(model, "김승희가 영화 더 잘 만들듯ㅋ")

0.10448114573955536

In [76]:
predict_sentiment(model, "결말이 참 인상깊은 영화였습니다. 따봉드립니다.")

0.9945418238639832

In [66]:
predict_sentiment(model, "개지린다")

0.5397792458534241